In [410]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import metrics 
from sklearn import svm

In [411]:
#read in the data using pandas
df1 = pd.read_csv(Path.cwd().parent / "data" / "07_final_dataset.csv")
df2 = pd.read_csv(Path.cwd().parent / "data" / "05_dataset_before_corr_mat.csv")
to_drop1 = [col for col in df1.columns if "Unnamed" in col.split(":")]
to_drop2 = [col for col in df2.columns if "Unnamed" in col.split(":")] + ["artist_genres"]
df1 = df1.drop(columns=to_drop1)
df2 = df2.drop(columns=to_drop2)
print(df1.shape, df2.shape)

(4628, 26) (4628, 11)


In [412]:
df1.head(10)

,artist_popularity,artist_followers,track_popularity,track_danceability,track_energy,track_loudness,track_tempo,track_duration_ms,is_explicit_content,words_count,...,house,metal,soul,r&b,electro,hip hop,funk,folk,disco,other_genre
0,86,2352780,90,0.761,0.525,-6.900,80.870,238805,False,411.0,...,0,0,0,0,0,0,0,0,0,1
1,90,3212078,97,0.591,0.764,-5.484,169.928,141806,True,606.0,...,0,0,0,0,0,0,0,0,0,1
2,90,8574878,96,0.796,0.798,-6.312,116.032,202735,False,280.0,...,0,0,0,0,0,0,0,0,0,1
3,94,33127866,97,0.604,0.366,-7.519,141.981,224695,False,208.0,...,0,0,0,0,0,0,0,0,0,0
4,100,40894024,92,0.735,0.795,-6.523,122.000,188918,True,283.0,...,0,0,0,0,0,0,0,0,0,0
5,96,91127160,44,0.788,0.859,-2.724,141.020,207853,False,519.0,...,0,0,0,0,0,0,0,0,0,0
6,84,401771,94,0.727,0.739,-3.805,93.005,145800,True,240.0,...,0,0,0,0,0,0,0,0,0,1
7,90,8958365,96,0.736,0.704,-7.409,149.995,212000,True,613.0,...,0,0,0,0,0,0,0,0,0,0
8,81,740641,94,0.671,0.673,-6.941,121.963,237720,False,231.0,...,0,0,0,0,0,0,0,0,0,0
9,92,37817210,98,0.728,0.783,-4.424,77.011,173381,False,449.0,...,0,0,0,0,0,0,0,0,0,0


In [413]:
to_drop = []
for i, row in df1.iterrows():
    if row["country_code"][0].isalpha():
        to_drop.append(i)

In [414]:
df1.drop(index=to_drop, axis=0, inplace=True)
df2.drop(index=to_drop, axis=0, inplace=True)

In [415]:
# usa = 1 else = 0
df1.loc[df1["country_code"] == "840", "country_code"] = 1
df1.loc[df1["country_code"] != "840", "country_code"] = 0
df2.loc[df2["country_code"] == "840", "country_code"] = 1
df2.loc[df2["country_code"] != "840", "country_code"] = 0
df1["country_code"] = df1["country_code"].astype(bool).astype("category")
df2["country_code"] = df2["country_code"].astype(bool).astype("category")

In [416]:
df1["artist_name"] = df1["artist_name"].astype("category")
df2["artist_name"] = df2["artist_name"].astype("category")

In [417]:
med = df1["words_count"].median()
for j, row in df1.iterrows():
     df1.loc[j, "words_count"] = 1 if df1.loc[j, "words_count"] > med else 0
df1["words_count"] = df1["words_count"].astype(bool).astype("category")

In [418]:
df1.head(10)

,artist_popularity,artist_followers,track_popularity,track_danceability,track_energy,track_loudness,track_tempo,track_duration_ms,is_explicit_content,words_count,...,house,metal,soul,r&b,electro,hip hop,funk,folk,disco,other_genre
0,86,2352780,90,0.761,0.525,-6.900,80.870,238805,False,True,...,0,0,0,0,0,0,0,0,0,1
1,90,3212078,97,0.591,0.764,-5.484,169.928,141806,True,True,...,0,0,0,0,0,0,0,0,0,1
2,90,8574878,96,0.796,0.798,-6.312,116.032,202735,False,False,...,0,0,0,0,0,0,0,0,0,1
3,94,33127866,97,0.604,0.366,-7.519,141.981,224695,False,False,...,0,0,0,0,0,0,0,0,0,0
4,100,40894024,92,0.735,0.795,-6.523,122.000,188918,True,False,...,0,0,0,0,0,0,0,0,0,0
5,96,91127160,44,0.788,0.859,-2.724,141.020,207853,False,True,...,0,0,0,0,0,0,0,0,0,0
6,84,401771,94,0.727,0.739,-3.805,93.005,145800,True,False,...,0,0,0,0,0,0,0,0,0,1
7,90,8958365,96,0.736,0.704,-7.409,149.995,212000,True,True,...,0,0,0,0,0,0,0,0,0,0
8,81,740641,94,0.671,0.673,-6.941,121.963,237720,False,False,...,0,0,0,0,0,0,0,0,0,0
9,92,37817210,98,0.728,0.783,-4.424,77.011,173381,False,True,...,0,0,0,0,0,0,0,0,0,0


# Remove Outliers

In [419]:
# # take all numerics
# quatiative = df.select_dtypes(exclude=['category'])

# # boundaries for chaking outliers
# upper_bound = quatiative.mean() + 3 * quatiative.std()
# lower_bound = quatiative.mean() - 3 * quatiative.std()

# outliers = []

# for num, row in quatiative.iterrows():
#     outliers_in_row = [1 for col in quatiative.columns if row[col] > upper_bound[col] or row[col] < lower_bound[col]].count(1)    
#     # add this index(row) if 30% of it's content is outlaiered
#     if outliers_in_row > 0.1 * len(row):
#         outliers.append(num)

# print(f"Number of indexes with 30% outliers of it's content are: {len(outliers)}")

In [420]:
# df.drop(outliers, inplace=True)

# df.shape

In [421]:
df1["is_explicit_content"] = df1["is_explicit_content"].astype("int").astype("category")
df2["is_explicit_content"] = df2["is_explicit_content"].astype("int").astype("category")

In [422]:
df2.dtypes

artist_popularity         int64
artist_followers          int64
track_popularity          int64
track_danceability      float64
track_energy            float64
track_loudness          float64
track_tempo             float64
track_duration_ms         int64
is_explicit_content    category
words_count             float64
country_code           category
dtype: object

# Normalization

In [423]:
# df.describe()

In [424]:
# from sklearn.preprocessing import MinMaxScaler
# cols_to_scale = ["artist_popularity","artist_followers","track_popularity", "track_tempo", "track_loudness", "track_duration_ms"]
# for c in cols_to_scale:
#     scaler = MinMaxScaler() 
#     scaled_values = scaler.fit_transform(df[c].to_numpy().reshape(-1, 1)) 
#     df.loc[:,c] = scaled_values

In [425]:
# df.describe()

In [426]:
df1.head(10)

,artist_popularity,artist_followers,track_popularity,track_danceability,track_energy,track_loudness,track_tempo,track_duration_ms,is_explicit_content,words_count,...,house,metal,soul,r&b,electro,hip hop,funk,folk,disco,other_genre
0,86,2352780,90,0.761,0.525,-6.900,80.870,238805,0,True,...,0,0,0,0,0,0,0,0,0,1
1,90,3212078,97,0.591,0.764,-5.484,169.928,141806,1,True,...,0,0,0,0,0,0,0,0,0,1
2,90,8574878,96,0.796,0.798,-6.312,116.032,202735,0,False,...,0,0,0,0,0,0,0,0,0,1
3,94,33127866,97,0.604,0.366,-7.519,141.981,224695,0,False,...,0,0,0,0,0,0,0,0,0,0
4,100,40894024,92,0.735,0.795,-6.523,122.000,188918,1,False,...,0,0,0,0,0,0,0,0,0,0
5,96,91127160,44,0.788,0.859,-2.724,141.020,207853,0,True,...,0,0,0,0,0,0,0,0,0,0
6,84,401771,94,0.727,0.739,-3.805,93.005,145800,1,False,...,0,0,0,0,0,0,0,0,0,1
7,90,8958365,96,0.736,0.704,-7.409,149.995,212000,1,True,...,0,0,0,0,0,0,0,0,0,0
8,81,740641,94,0.671,0.673,-6.941,121.963,237720,0,False,...,0,0,0,0,0,0,0,0,0,0
9,92,37817210,98,0.728,0.783,-4.424,77.011,173381,0,True,...,0,0,0,0,0,0,0,0,0,0


In [427]:
X1 = df1.drop(columns=["is_explicit_content"])
y1 = df1["is_explicit_content"]
X2 = df2.drop(columns=["is_explicit_content"])
y2 = df2["is_explicit_content"]

# All in one

In [428]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=1)
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=1)

In [429]:
X1.head(10)

,artist_popularity,artist_followers,track_popularity,track_danceability,track_energy,track_loudness,track_tempo,track_duration_ms,words_count,country_code,...,house,metal,soul,r&b,electro,hip hop,funk,folk,disco,other_genre
0,86,2352780,90,0.761,0.525,-6.900,80.870,238805,True,False,...,0,0,0,0,0,0,0,0,0,1
1,90,3212078,97,0.591,0.764,-5.484,169.928,141806,True,False,...,0,0,0,0,0,0,0,0,0,1
2,90,8574878,96,0.796,0.798,-6.312,116.032,202735,False,False,...,0,0,0,0,0,0,0,0,0,1
3,94,33127866,97,0.604,0.366,-7.519,141.981,224695,False,False,...,0,0,0,0,0,0,0,0,0,0
4,100,40894024,92,0.735,0.795,-6.523,122.000,188918,False,False,...,0,0,0,0,0,0,0,0,0,0
5,96,91127160,44,0.788,0.859,-2.724,141.020,207853,True,False,...,0,0,0,0,0,0,0,0,0,0
6,84,401771,94,0.727,0.739,-3.805,93.005,145800,False,False,...,0,0,0,0,0,0,0,0,0,1
7,90,8958365,96,0.736,0.704,-7.409,149.995,212000,True,False,...,0,0,0,0,0,0,0,0,0,0
8,81,740641,94,0.671,0.673,-6.941,121.963,237720,False,False,...,0,0,0,0,0,0,0,0,0,0
9,92,37817210,98,0.728,0.783,-4.424,77.011,173381,True,False,...,0,0,0,0,0,0,0,0,0,0


In [430]:
from sklearn.model_selection import GridSearchCV

params_decision_tree = {"max_depth": [7,9,15], "min_samples_split": [3,5,7,10]}
params_random_forest = {"n_estimators": [150,200, 250],"max_depth": [7,11,15]}
params_knn = {"n_neighbors": range(3, 20)}

clf_params = [(DecisionTreeClassifier(), params_decision_tree),
             (RandomForestClassifier(), params_random_forest), 
             (KNeighborsClassifier(), params_knn)]
             

for idx, (clf, params) in enumerate(clf_params):
    clf_cv = GridSearchCV(clf,params,cv=10)
    clf_cv.fit(X1,y1)
    print(alg_names[idx])
    print("==========================")
    print(f"best params are: {clf_cv.best_params_}")
    print(f"best score is: {clf_cv.best_score_}")

decision tree
best params are: {'max_depth': 7, 'min_samples_split': 7}
best score is: 0.821201970198881
random forest
best params are: {'max_depth': 11, 'n_estimators': 250}
best score is: 0.8607839853103559
KNN
best params are: {'n_neighbors': 6}
best score is: 0.7205921762757287


In [431]:
from sklearn.model_selection import GridSearchCV

params_decision_tree = {"max_depth": [7,9,15], "min_samples_split": [3,5,7,10]}
params_random_forest = {"n_estimators": [150,200, 250],"max_depth": [7,11,15]}
params_knn = {"n_neighbors": range(3, 20)}

clf_params = [(DecisionTreeClassifier(), params_decision_tree),
             (RandomForestClassifier(), params_random_forest), 
             (KNeighborsClassifier(), params_knn)]
             

for idx, (clf, params) in enumerate(clf_params):
    clf_cv = GridSearchCV(clf,params,cv=10)
    clf_cv.fit(X2,y2)
    print(alg_names[idx])
    print("==========================")
    print(f"best params are: {clf_cv.best_params_}")
    print(f"best score is: {clf_cv.best_score_}")

decision tree
best params are: {'max_depth': 7, 'min_samples_split': 10}
best score is: 0.7985533678242206
random forest
best params are: {'max_depth': 15, 'n_estimators': 200}
best score is: 0.8256901279140457
KNN
best params are: {'n_neighbors': 6}
best score is: 0.7210896887135395
